In [8]:
import os

import yaml
from box import Box

import torch
from transformers import (
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    AutoTokenizer,
)

import numpy as np
from transformers import LlamaTokenizer
from transformers import pipeline
from peft import get_peft_model, PeftConfig, PeftModel

In [2]:
with open("config.yml", "r") as file:
    config = Box(yaml.safe_load(file))

os.environ["HF_TOKEN"] = config.hf_token

In [3]:
tokenizer = AutoTokenizer.from_pretrained(
    config.lora_path,
    trust_remote_code=True,
)

# tokenizer.pad_token = tokenizer.eos_token
# max_seq_length = tokenizer.model_max_length

In [4]:
model = AutoModelForCausalLM.from_pretrained(
    config.model_name,
    token=config.hf_token,
    load_in_8bit=False,
    torch_dtype=torch.bfloat16,
    device_map="cuda"
)

Loading checkpoint shards: 100%|██████████| 5/5 [00:07<00:00,  1.41s/it]


In [5]:
model.resize_token_embeddings(
    len(tokenizer),
    mean_resizing=False
)

Embedding(131074, 5120)

In [6]:
model.tie_weights()

In [10]:
model = PeftModel.from_pretrained(model, config.lora_path)

In [ ]:
# model = model.cuda()

In [24]:
def generate_text(model: AutoModelForCausalLM,
                  tokenizer: AutoTokenizer,
                  query: str,
                  device='cuda') -> str:
    """ Tokenize query, pass it to the model,
    convert tokens to human language text and return it
    """
    chat = [
        {"role": "user", "content": query},
    ]

    inputs = tokenizer.apply_chat_template(
        chat,
        return_tensors="pt",
        truncation=True,
        # padding="max_length",
        max_length=512,
        return_dict=True,
        add_generation_prompt=True
    ).to(device)
    
    # attention_mask = (
    #     inputs["input_ids"] != tokenizer.pad_token_id
    # ).long().to(device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            # attention_mask=attention_mask,
            # max_length=100,
            max_new_tokens=200,
            num_return_sequences=1,
            temperature=0.3
        )

    generated_text = tokenizer.decode(
        outputs[0],
        skip_special_tokens=True
    )

    generated_text = generated_text[len(query):]    

    return generated_text

In [27]:
query = """Your task is to answer the user's questions using only the information provided in the documents. Be sure to include a list of relevant documents ids that you used to formulate your answer.Context:{"content": [{"doc_id": "666", "content": "В 2024 году американские выборы выиграл Трамп"}, {"doc_id": "1312", "content": "В 2020 году американские выборы выиграл Байден"}]}
Question:кто выиграл американские выборы в 2034 году?"""

print(query)

Your task is to answer the user's questions using only the information provided in the documents. Be sure to include a list of relevant documents ids that you used to formulate your answer.Context:{"content": [{"doc_id": "666", "content": "В 2024 году американские выборы выиграл Трамп"}, {"doc_id": "1312", "content": "В 2020 году американские выборы выиграл Байден"}]}
Question:кто выиграл американские выборы в 2034 году?


In [28]:
%%time

generate_text(
    model=model,
    tokenizer=tokenizer,
    query=query
)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


CPU times: user 4.97 s, sys: 7.83 ms, total: 4.98 s
Wall time: 4.97 s


' году?assistant\n\nК сожалению, я не могу найти информацию о том, кто выиграл американские выборы в 2034 году, используя данные, доступные мне. Моя способность отвечать ограничена информацией, которую я могу найти в базе знаний.'

In [ ]:
# config = PeftConfig.from_pretrained(config.lora_path)
# model = get_peft_model(model, config)
# model = model.cuda()

In [ ]:
model_id = "mistralai/Mistral-Nemo-Instruct-2407"
lora_path = "models/sft-grndmrag-mistral-nemo-lora-128-2/checkpoint-10200"# "mistralai/Mistral-Nemo-Instruct-2407"
tokenizer = AutoTokenizer.from_pretrained(lora_path)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    load_in_8bit=False,
    torch_dtype="auto",
    device_map="auto",
)

model.resize_token_embeddings(len(tokenizer))
model.tie_weights()
model = PeftModel.from_pretrained(model, lora_path)
